In [ ]:
### Imports from algo_common

import csv
import collections
import datetime
from dateutil import tz
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import numpy as np
import os
import pandas as pd
import pytz
import sys
import time

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils
from UnitTests import testutils


In [ ]:
sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 

In [ ]:
context = Utils.initializeJupyter(sim)

In [ ]:
# load ema data
logName = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\Daily_Log_2022-11-17.csv"
logColumns = ["time", "fast", "fast1", "slow", "tLine", "macd", "signalLine", "longBool", "shortBool", "longFlag", "shortFlag", 
    "enterFlag", "entryFlag", "exitFlag", "askPrice", "bidPrice", "lastPrice", "midPrice", "portfolioValue", "positionValue", "cash"]
logData = pd.read_csv(logName, header=1, names=logColumns)
logData = Utils.cleanLogData(logData)

logData['date'] = np.array([datetime.datetime.fromtimestamp(float(f)) for f in logData.time.values])


In [ ]:
logData1 = logData[logData.enterFlag==True]
logData2 = logData1[logData1.entryFlag==True]

In [ ]:
plt.plot(np.arange(0,len(logData.entryFlag.values)),logData.entryFlag)
plt.xlim((24000,26000))

In [ ]:
# implement counter

# trigger to increase counter
# store entry flag data for last N minutes (start with 15 for now)
# check if entry flag flipped M times in last N minutes (start with 3 for now)
# if so, hold position unless X happens (talk with Mike about it)

In [ ]:
def appendCounter(context, N: int = 15):
    """ This function takes in a counter vector and throws out the left side and appends the right side if the vector is 
    greater than size N"""
    context.counter.append(context.entry_flag)
    if len(context.counter)>=N:
        context.counter.popleft()
    
def countFlips(context):
    """ This function takes in an input vector and counts the number of times the vector has flipped from true to false"""
    return np.count_nonzero(np.array([context.counter[i]!=context.counter[i+1] for i in range(len(context.counter)-1)]))
    

In [ ]:
flagCheck = collections.deque()
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(False)
flagCheck.append(False)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(True)
flagCheck.append(False)
flagCheck.append(False)
flagCheck.append(True)
flagCheck.append(True)

print(flagCheck)

In [ ]:
flagCheck.popleft()
flagCheck.append(False)
print(len(flagCheck))
print(flagCheck)

In [ ]:
counts = np.count_nonzero(np.array([flagCheck[i]!=flagCheck[i+1] for i in range(len(flagCheck)-1)]))
print(counts)

In [ ]:
[flagCheck[i]!=flagCheck[i+1] for i in range(len(flagCheck)-1)]

In [ ]:
context.entry_flag = False

IndicatorCalculations.appendCounter(context)
print(context.counter)
print(IndicatorCalculations.countFlips(context))
print(len(context.counter))

In [ ]:
def test_counterFunctions():
    sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 
    context = Utils.initializeJupyter(sim)
    choice = [True, False, True, True, True, False, False, True, True, False, False, True, True, False, True, True, False, True]
    output = [True, True, False, False, True, True, False, False, True, True, False, True, True, False, True]
    N=15

    for i in range(len(choice)):
        context.entry_flag = choice[i]
        IndicatorCalculations.appendCounter(context,N=N)

    assert IndicatorCalculations.countFlips(context)==8, "wrong number of flips counted"
    assert len(context.counter)==N, "Counter vector is the wrong length"
    assert np.all([context.counter[i]==output[i] for i in range(len(output))]), "Output Vector does not match"
    
test_counterFunctions()

In [ ]:
context.counter